In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50, EfficientNetB0, resnet50, efficientnet
from tensorflow.keras.optimizers import Adam

# ---------------------------------------------------
# 1. Load the Caltech-101 Dataset
# ---------------------------------------------------
# Create train/test split
(dataset_train, dataset_test), info = tfds.load(
    'caltech101',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

# Number of classes in Caltech-101
num_classes = info.features['label'].num_classes

# ---------------------------------------------------
# 2. Preprocess Data for ResNet and EfficientNet
# ---------------------------------------------------
IMG_SIZE = 224
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_resnet(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = resnet50.preprocess_input(image)
    return image, tf.one_hot(label, num_classes)

def preprocess_efficientnet(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = efficientnet.preprocess_input(image)
    return image, tf.one_hot(label, num_classes)

# Create tf.data pipelines
train_ds_resnet = (dataset_train
                   .map(preprocess_resnet, num_parallel_calls=AUTOTUNE)
                   .batch(batch_size)
                   .prefetch(AUTOTUNE))

val_ds_resnet = (dataset_test
                 .map(preprocess_resnet, num_parallel_calls=AUTOTUNE)
                 .batch(batch_size)
                 .prefetch(AUTOTUNE))

train_ds_eff = (dataset_train
                .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE)
                .batch(batch_size)
                .prefetch(AUTOTUNE))

val_ds_eff = (dataset_test
              .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE)
              .batch(batch_size)
              .prefetch(AUTOTUNE))

# ---------------------------------------------------
# 3. Define ResNet50 Model Without Gating
# ---------------------------------------------------
def create_resnet(input_shape, num_classes):
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=input_shape)
    base_model.trainable = True

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs=base_model.input, outputs=outputs, name="ResNet")
    return model

# ---------------------------------------------------
# 4. Define EfficientNetB0 Model Without Gating
# ---------------------------------------------------
def create_efficientnet(input_shape, num_classes):
    base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=input_shape)
    base_model.trainable = True

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs=base_model.input, outputs=outputs, name="EfficientNet")
    return model

# ---------------------------------------------------
# 5. Compile and Train the Models
# ---------------------------------------------------
# ResNet50 Model
resnet_model = create_resnet((IMG_SIZE, IMG_SIZE, 3), num_classes)
resnet_model.compile(optimizer=Adam(learning_rate=1e-4), loss="categorical_crossentropy", metrics=["accuracy"])

print("\n--- Fine-Tuning ResNet50 on Caltech-101 ---")
history_resnet = resnet_model.fit(train_ds_resnet, epochs=3, validation_data=val_ds_resnet, verbose=1)

# EfficientNetB0 Model
eff_model = create_efficientnet((IMG_SIZE, IMG_SIZE, 3), num_classes)
eff_model.compile(optimizer=Adam(learning_rate=1e-4), loss="categorical_crossentropy", metrics=["accuracy"])

print("\n--- Fine-Tuning EfficientNetB0 on Caltech-101 ---")
history_eff = eff_model.fit(train_ds_eff, epochs=3, validation_data=val_ds_eff, verbose=1)

# Evaluate ResNet50
loss_resnet, acc_resnet = resnet_model.evaluate(val_ds_resnet, verbose=0)
print(f"ResNet50 Final Accuracy: {acc_resnet:.4f}")

# Evaluate EfficientNetB0
loss_eff, acc_eff = eff_model.evaluate(val_ds_eff, verbose=0)
print(f"EfficientNetB0 Final Accuracy: {acc_eff:.4f}")


--- Fine-Tuning ResNet50 on Caltech-101 ---
Epoch 1/3
77/77 ━━━━━━━━━━━━━━━━━━━━ 93s 603ms/step - accuracy: 0.1736 - loss: 4.2374 - val_accuracy: 0.8056 - val_loss: 0.7732
Epoch 2/3
77/77 ━━━━━━━━━━━━━━━━━━━━ 24s 305ms/step - accuracy: 0.9603 - loss: 0.4091 - val_accuracy: 0.8987 - val_loss: 0.4160
Epoch 3/3
77/77 ━━━━━━━━━━━━━━━━━━━━ 24s 317ms/step - accuracy: 0.9983 - loss: 0.0539 - val_accuracy: 0.9167 - val_loss: 0.3838

--- Fine-Tuning EfficientNetB0 on Caltech-101 ---
Epoch 1/3
77/77 ━━━━━━━━━━━━━━━━━━━━ 107s 643ms/step - accuracy: 0.0605 - loss: 4.4885 - val_accuracy: 0.5425 - val_loss: 3.4886
Epoch 2/3
77/77 ━━━━━━━━━━━━━━━━━━━━ 11s 136ms/step - accuracy: 0.6007 - loss: 3.1051 - val_accuracy: 0.8039 - val_loss: 1.9104
Epoch 3/3
77/77 ━━━━━━━━━━━━━━━━━━━━ 11s 138ms/step - accuracy: 0.8644 - loss: 1.6996 - val_accuracy: 0.8725 - val_loss: 0.9881
ResNet50 Final Accuracy: 0.9167
EfficientNetB0 Final Accuracy: 0.8725
